In [317]:
import numpy as np
from sklearn import linear_model
import pandas
from sklearn.metrics import mean_squared_error
from sklearn import ensemble
from sklearn.cross_validation import train_test_split

In [318]:
def convert_feature_to_dummy(rows, number_of_possible_values, converter_func):
    number_of_rows = len(rows)
    dummies = np.zeros((number_of_rows, number_of_possible_values))
    for i in xrange(number_of_rows):
        dummies[i, converter_func(rows[i])] = 1
    return dummies

In [319]:
def time_converter(row):
    return int(row)

In [320]:
def day_of_month_converter(row):
    return int(row)

In [321]:
def day_of_week_converter(row):
    return int(row)

In [322]:
def zone_converter(row):
    return int(row)

In [323]:
def my_hstack(args):
    return reduce(lambda x, y: np.hstack((x, y)), args)

In [380]:
df = pandas.read_csv("all.csv", names=["YearDay", "DayOfWeek", "Hour", "RushHour", 
                                       "GoodWeather", "IsRain", "Holiday", "Event1", 
                                       "Event2", "Requests", "Zone", "PriceFeature", "PolicyApplied"])

In [387]:
df.head(5)

,YearDay,DayOfWeek,Hour,RushHour,GoodWeather,IsRain,Holiday,Event1,Event2,Requests,Zone,PriceFeature,PolicyApplied
0,0,6,0,0,1,0,0,0,1,34,0,0,0
1,0,6,0,0,1,0,0,0,1,35,1,0,0
2,0,6,0,0,1,0,0,0,1,31,2,0,0
3,0,6,0,0,1,0,0,0,1,38,3,0,0
4,0,6,0,0,1,0,0,0,1,30,4,0,0


In [363]:
year_day = []
day_of_week = []
hour = []
zone = []
rush_hour = []
good_weather = []
# raining = []
holiday = []
feature1 = []
feature2 = []
event1 = []
event2 = []
requests = []
for index, row in df.iterrows():
    year_day.append(row['YearDay'])
    day_of_week.append(row['DayOfWeek'])
    hour.append(row['Hour'])
    zone.append(row['Zone'])
    rush_hour.append([row['RushHour']])
    good_weather.append([row['GoodWeather']])
    holiday.append([row['Holiday']])
    event1.append([row['Event1']])
    event2.append([row['Event2']])
    feature1.append([row["PriceFeature"]])
    feature2.append([row["PolicyApplied"]])
    requests.append(row['Requests'])

In [369]:
#year_day_dummy = convert_feature_to_dummy(year_day, 365, day_of_month_converter) 
day_of_week_dummy = convert_feature_to_dummy(day_of_week, 7, day_of_week_converter)
hour_dummy = convert_feature_to_dummy(hour, 24, time_converter)
zone_dummy = convert_feature_to_dummy(zone, 8, zone_converter)

In [372]:
result = my_hstack([day_of_week_dummy, hour_dummy, zone_dummy, rush_hour, good_weather, 
                    holiday, feature1, feature2, event1, event2])

In [373]:
params = {'n_estimators': 1000, 'max_depth': 2, 'min_samples_split': 2,
          'learning_rate': 0.01, 'loss': 'lad'}
clf = ensemble.GradientBoostingRegressor(**params)

In [378]:
X_train, X_test, y_train, y_test = train_test_split(result, requests, test_size=0.2, random_state=42)

In [379]:
clf.fit(X_test, y_test)
mean_squared_error(y_test, clf.predict(X_test))

393.65532248858449

In [407]:
res_df = pandas.concat([df.head(196)['Hour'], df.head(196)['Zone'],df.head(196)['Requests']], axis=1)


In [410]:
res_df.to_csv("predict.csv", header=False,index=False)

In [409]:
res_df

,Hour,Zone,Requests
0,0,0,34
1,0,1,35
2,0,2,31
3,0,3,38
4,0,4,30
5,0,5,35
6,0,6,38
7,0,7,44
8,1,0,48
9,1,1,57
